In [1]:
import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import create_aggregate_df as agg_df

%matplotlib inline
pd.set_option('display.max_columns', None);

In [4]:
#read in accepted loan data
path = "../data/2012_13_loan_data/LoanStats3b.csv";
loans_2013_all = pd.read_csv(path, sep=',', low_memory = False, encoding='latin-1');

# loans_2013_all = agg_df.create_aggregate_df(path);
display(loans_2013_all.head());

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,10000,10000,10000.0,36 months,9.67%,321.13,B,B1,Registered Nurse,7 years,MORTGAGE,102000.0,Not Verified,Dec-13,Fully Paid,n,NaN,NaN,debt_consolidation,Clean Up,027xx,MA,15.55,2,Oct-89,0,11.0,NaN,9,0,9912,44.40%,22,f,0.0,0.0,11560.462180,11560.46,10000.0,1560.46,0.0,0.0,0.0,Jan-17,320.91,NaN,Dec-16,0,54.0,1,Individual,NaN,NaN,NaN,0,0,39143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22300,NaN,NaN,NaN,3,4349.0,973.0,89.4,0,0,243.0,290,23,8,0,25.0,11.0,8.0,11.0,1,3,4,3,6,9,6,13,4,9,0.0,0,0,1,77.3,66.7,0,0,58486,39143,9200,36186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,12000,12000,12000.0,36 months,6.62%,368.45,A,A2,MANAGER INFORMATION DELIVERY,10+ years,MORTGAGE,105000.0,Not Verified,Dec-13,Fully Paid,n,NaN,NaN,debt_consolidation,UNIVERSAL CARD,060xx,CT,14.05,0,Mar-94,1,43.0,NaN,12,0,13168,21.60%,22,w,0.0,0.0,13263.954640,13263.95,12000.0,1263.95,0.0,0.0,0.0,Jan-17,368.20,NaN,Oct-18,0,NaN,1,Individual,NaN,NaN,NaN,0,0,267646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61100,NaN,NaN,NaN,4,26765.0,39432.0,25.0,0,0,146.0,237,20,3,4,20.0,NaN,3.0,43.0,0,2,2,5,5,9,8,9,2,12,0.0,0,0,2,95.5,0.0,0,0,333044,42603,52600,42769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,20800,20800,20800.0,36 months,13.53%,706.16,B,B5,Operations Manager,10+ years,RENT,81500.0,Verified,Dec-13,Fully Paid,n,NaN,Borrower added on 12/31/13 > My goal is to p...,debt_consolidation,Reducing Debt to Purchase Home,100xx,NY,16.73,0,Jun-98,2,64.0,NaN,29,0,23473,54.50%,41,f,0.0,0.0,23926.640010,23926.64,208

In [3]:
loans_2013_all.shape

(235629, 145)

In [5]:
#remove current loans
loans_2013_all = loans_2013_all.loc[loans_2013_all.loan_status != 'Current'];

In [6]:
#check the policy code for all loans that are not 'Current'
print(loans_2013_all.policy_code.unique())

[1]


In [7]:
#make category for whether the loan defaulted
loans_2013_all['default'] = 1;
loans_2013_all.loc[loans_2013_all.loan_status == 'Fully Paid', 'default'] = 0;

In [8]:
#calculate the def
ratio = loans_2013_all.default.sum() / loans_2013_all.default.count()
print('Default Percentage = {0}'.format(ratio))

Default Percentage = 0.15822457051508168


In [5]:
#drop all of the hardship values
cutoff = np.where( loans_2013_all.columns.values == 'hardship_flag' )[0][0];
loans_2013_all_pred_subset = loans_2013_all.iloc[:, 0:cutoff];


loans_2013_all_pred_subset = loans_2013_all_pred_subset.drop(labels=['id', #id column
                                                         'member_id', #member_id
                                                         'funded_amnt', #funded amount so far
                                                         'funded_amnt_inv', #funded amount by investors so far
                                                         'url', #url of LC listing page
                                                         'desc', #description of what loan is for
                                                         'title', #title for loan
                                                         'pymnt_plan', #is there a payment plan- y/n?
                                                         'last_pymnt_d', #the last payment date
                                                         'last_pymnt_amnt', #last payment amount
                                                         'next_pymnt_d', #next payment amount
                                                         'total_rec_int', #Interest received to date
                                                         'total_rec_late_fee', #Late fees received to date
                                                         'total_rec_prncp', #Principal received to date
                                                         'emp_title' #employee title
                                                                    ], axis=1);


In [6]:
print(loans_2013_all_pred_subset.shape)
display(loans_2013_all_pred_subset.head())

(218501, 107)


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,recoveries,collection_recovery_fee,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,10400.0,36 months,6.99%,321.08,A,A3,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,credit_card,937xx,CA,14.92,0.0,Sep-1989,2.0,42.0,NaN,17.0,0.0,6133.0,31.6%,36.0,w,0.0,0.0,6611.69,6611.69,521.27,93.8286,Feb-2017,0.0,59.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,162110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19400.0,NaN,NaN,NaN,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,42.0,1.0,42.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15000.0,60 months,12.39%,336.64,C,C1,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,debt_consolidation,235xx,VA,12.03,0.0,Aug-1994,0.0,NaN,NaN,6.0,0.0,138008.0,29%,17.0,w,0.0,0.0,17392.37,17392.37,0.00,0.0000,Nov-2017,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,149140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0,NaN,NaN,NaN,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,NaN,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7650.0,36 months,13.66%,260.20,C,C3,< 1 year,RENT,50000.0,Source Verified,Dec-2014,Charged Off,debt_consolidation,850xx,AZ,34.81,0.0,Aug-2002,1.0,NaN,NaN,11.0,0.0,16822.0,91.9%,20.0,f,0.0,0.0,2281.98,2281.98,1237.99,222.8382,Oct-2016,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,64426.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18300.0,NaN,NaN,NaN,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,0.0,17.0,NaN,3.0,NaN,0.0,1.0,4.0,1.0,4.0,12.0,4.0,8.0,4.0,11.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,82331.0,64426.0,4900.0,64031.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9600.0,36 months,13.66%,326.53,C,C3,10+ years,RENT,69000.0,Source Verified,Dec-2014,Fully Paid,debt_consolidation,077xx,NJ,25.81,0.0,Nov-1992,0.0,NaN,NaN,12.0,0.0,16388.0,59.4%,44.0,f,0.0,0.0,9973.43,9973.43,0.00,0.0000,Oct-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,38566.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27600.0,NaN,NaN,NaN,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,NaN,17.0,NaN,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21425.0,60 months,15.59%,516.3

In [13]:
loans_2013_all_pred_subset[loans_2013_all_pred_subset.policy_code.isnull()].shape

(2, 107)

Only 2 rows have `nan` for `policy_code`, so just I will  assume that these should be `1.0` instead of `nan`, so I'm dropping the `policy_code` column

In [15]:
#drop policy code
loans_2013_all_pred_subset = loans_2013_all_pred_subset.drop(labels=['policy_code'], axis=1);

loans_2013_all_pred_subset.shape

(218501, 106)

In [16]:
loans_2013_all_pred_subset.to_csv('all_2013_loans_predictor_subsets.csv');